In [ ]:
# URL and save directory
url = "https://www.youtube.com/watch?v=KwR3nxojS0g"
save_dir = "docs/youtube/"

# Function to generate transcript filename from URL
def generate_transcript_filename(url):
    parsed_url = urllib.parse.urlparse(url)
    video_id = urllib.parse.parse_qs(parsed_url.query).get('v', [None])[0]
    if video_id is None:
        raise ValueError("Unable to extract video ID from URL")
    return os.path.join(save_dir, f"transcript_{video_id}.json")

transcript_filename = generate_transcript_filename(url)

# Check if transcript already exists
if os.path.exists(transcript_filename):
    # Load the existing transcript
    with open(transcript_filename, 'r', encoding='utf-8') as file:
        docs = json.load(file)
else:
    # Download video, create transcript, and save it
    loader = GenericLoader(
        YoutubeAudioLoader([url], save_dir),
        OpenAIWhisperParser()
    )
    docs = loader.load()

    # Convert docs to a JSON-compatible format
    docs_json = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in docs]

    # Save the transcript and metadata as JSON
    with open(transcript_filename, 'w', encoding='utf-8') as file:
        json.dump(docs_json, file, ensure_ascii=False, indent=4)

In [ ]:
for doc in docs:
    print(doc["page_content"])